## PREDICCION DE NOTAS UTILIZANDO APRENDISAJE AUTOMATICO

In [118]:
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import textacy.preprocessing as tprep
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings


--------------------------------Cargo la base de datos---------------------------------------------------------------------

In [119]:
df = pd.read_csv("IOrespuestas.csv")
pd.options.mode.chained_assignment = None  # default='warn'
warnings.filterwarnings('ignore')

-----------------------------Preparo el Corpus para el trabajo---------------------------------------------------------

In [120]:
# SELLECIONO LAS COLUMNAS,CAMBIO EL NOMBRE, ELIMINO LAS NA Y TRABAJO EL FORMATO

df_corpus=df[['Padrón sin números', '¿qué es la Investigación Operativa?']]
df_corpus.columns = ['padrón', 'texto']
df_corpus['notas'] = pd.read_csv(r"Resultado_Notas_queEsIO_aMano.csv")
df_corpus =df_corpus.dropna()
df_corpus['texto'] = df_corpus['texto'].astype('U')
df_corpus

,padrón,texto,notas
0,100723,Se basa en la utilización de métodos analític...,2
1,97739,"Optimización y diseño de operaciones, control ...",1
2,97920,Aplicaciones de métodos operativos a la vida real,0
3,102908,La optimización de las operaciones mediante el...,0
4,100586,Una forma interdisciplinaria de resolver probl...,0
...,...,...,...
115,100994,Es la utilización del metodo científico para e...,1
116,103787,Es el area que se encarga de aplicar distintos...,0
118,104911,Aplicación de la ciencia moderna a problemas c...,0
119,102939,No pude completar la encuesta con el celular e...,0


-------------------------------PREPROCESAMIENTO---------------------------------------------------------------------

In [121]:
wordnet_lematizer = WordNetLemmatizer()
spanish_stemmer = SnowballStemmer('spanish')

def preprocesar(texto):
    '''función que preprocesa el texto para ser trabajado'''
    
    tokens = [ tprep.remove.accents(str(w).lower()) for w in word_tokenize(texto) if w.isalpha() if len(w)>4]
    tokens = [ spanish_stemmer.stem(t) for t in tokens ]
    return tokens

--------------------------------Empiezo a entrenar---------------------------------------------------------------------

In [122]:
# DEFINO LOS DATOS DE ENTRENAMIENTO Y PRUEBA (80%/20%)
from sklearn.model_selection import train_test_split

x_train, x_test,y_train,y_test = train_test_split(df_corpus['texto'], df_corpus['notas'], test_size=0.2) 


In [123]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(tokenizer = preprocesar, stop_words=stopwords.words('spanish'))

In [124]:
#INSTANCIO LAS TRANSFORMADAS DE LA VARIABLES EXPLICATIVAS
count_train = cv.fit_transform(x_train)
count_test = cv.transform(x_test)

In [125]:
#USO DEL MODELO DE NAIVE-BAYES

from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

count_nb = MultinomialNB()
count_nb.fit(count_train, y_train)
count_nb_pred = count_nb.predict(count_test)
count_m = metrics.accuracy_score(y_test,count_nb_pred)


In [126]:
from sklearn.feature_extraction.text import TfidfVectorizer

tv = TfidfVectorizer(tokenizer = preprocesar, stop_words=stopwords.words('spanish') )
tfidf_train = tv.fit_transform(x_train.values.astype('U'))
tfidf_test = tv.transform(x_test.values.astype('U'))

In [127]:
# MIDO LA PRECISION DEL MODELO
f'La precisión de la predicción es del {round(count_m*100,2)}%' 

'La precisión de la predicción es del 83.33%'

In [128]:
#Realizo una prueba sobre un texto de ejemplo

texto_de_prueba = "Es la optimización mediante el modelado y aplicación de la matemática"

texto_de_prueba_vectorizado_cv = cv.transform([texto_de_prueba])
t_count_pred = count_nb.predict(texto_de_prueba_vectorizado_cv)
f'se predice la nota de "{int(t_count_pred) }" para el texto de prueba'


'se predice la nota de "1" para el texto de prueba'

### Resultados

In [129]:
def asignar_notas_ML(texto):
    '''función que asigna el modelo de Naive-Bayes usando texto y notas como variables'''
    notas_ML = [count_nb.predict(cv.transform([t])) for t in [texto] ]
    return int(sum(notas_ML))


In [130]:
df_corpus['notas_ML'] = df_corpus['texto'].apply(asignar_notas_ML)


In [131]:
df_corpus['diferencias'] = abs(df_corpus['notas']-df_corpus['notas_ML'])
# Total de diferencias
df_corpus[df_corpus['diferencias'] != 0]['notas_ML'].count()

11

In [132]:
df_corpus.to_excel(r"Resultado_Notas_queEsIO_ML.xlsx")